In [11]:
from CGRdb import load_schema
import os
from CGRtools.files import *

os.environ['PATH']+=':/opt/fragmentor'  # path to ISIDA fragmentor

# Create the database entity object

In [2]:
db = load_schema('db_name', user='user', password='pass', host='localhost', database='postgres')

/users/adelia/env/lib/python3.7/site-packages/LazyPony/__init__.py:80: UserWarning: for many-to-many relationship if schema used NEED to define m2m table name
  warn('for many-to-many relationship if schema used NEED to define m2m table name')


# Fill database

In [ ]:
from CGRtools.files import *
from pony.orm import db_session
with db_session:
    db.User(1, 'name')
    
reactions = RDFread('data.rdf')
molecules = SDFread('mol.sdf')
for reaction in reactions:
    db.Reaction(structure=reaction, user=db.User[1])

for molecule in molecules:
    db.Molecule(structure=molecule, user=db.User[1])
    

# Get all molecule and reaction entity objects

In [7]:
Molecules = db.Molecule
Reactions = db.Reaction

Get a molecular structure. If you need to get a SMILES representation use: print(structure)

In [8]:
molecule1 = Molecules[1]  # first molecule in db
structure = molecule1.structure  # molecular structure 
print(structure)
structure

C-C-C-C-C-Br


Get a reaction structure in SMIRKS representation.

In [9]:
reaction = Reactions[1]
structure = reaction.structure 
print(structure)
structure

[N+].[Cl-].C-C-C-C-C-Br>>C-C-C-C-C-C(=C)-C=C


If you need to store molecules/reactions from db into SDFile/RDFile:

In [12]:
with SDFwrite('molecules.sdf') as file:
    for molecule in Molecules.select()[:10]:
        file.write(molecule.structure)
        
with RDFwrite('reactions.rdf') as file:
    for reaction in Reactions.select()[:10]:
        file.write(reaction.structure)
        

# 3. Search 

   # Molecule Search
   Substructure and Similarity search returns list of tuples contains result molecule and Tanimoto coefficient. 
1. Structure search


In [31]:
query = SDFread('molecules.sdf').read()[7]
db.Molecule.find_structure(query)

Molecule[8]

2. Substructure search

In [16]:
substructures = db.Molecule.find_substructures(query)

In [17]:
substructures[:10] # list of tuples (Molecule, Tanimoto) ordered by Tanimoto

[(Molecule[8], 1.0),
 (Molecule[274062], 1.0),
 (Molecule[348007], 0.666666686534882),
 (Molecule[175109], 0.5),
 (Molecule[688526], 0.5),
 (Molecule[15646], 0.5),
 (Molecule[68990], 0.5),
 (Molecule[617562], 0.5),
 (Molecule[306425], 0.5),
 (Molecule[521019], 0.5)]

3. Similarity search

In [21]:
from pony.orm import db_session
with db_session:
    similar = db.Molecule.find_similar(query)

In [22]:
similar

[(Molecule[8], 1.0), (Molecule[274062], 1.0)]

    Save the search results into file

In [23]:
with SDFwrite('molecules_s.sdf') as file:
    for t in substructures:
        file.write(t[0].structure)
    

# Reaction by molecule search. 
Searching by reactions contains query molecule.
It is also possible to search reactions with molecule (the same, superstructure, similar) in proper role: reagent or product. 


In [32]:
# reactions with query in reagents/products
reactions_by_molecule = db.Molecule.find_reactions_by_molecule(query)  
# reactions with superstructure of query in reagents/products
reactions_with_substructures = db.Molecule.find_reaction_by_substructure_molecule(query)
# reactions with similar to query molecules in reagents/products
reactions_with_similar = db.Molecule.find_reaction_by_similar_molecule(query)
# reactions with query only in reagents
reactions_with_reagent = db.Molecule.find_reactions_by_reagent(query)
# reactions with query only in products
reactions_with_product = db.Molecule.find_reactions_by_product(query)
# reactions with superstructure of query only in reagents
reactions_with_substructure_reagent = db.Molecule.find_reactions_by_substructure_reagent(query)
# reactions with superstructure of query only in products
reactions_with_substructure_product = db.Molecule.find_reactions_by_substructure_product(query)
# reactions with similar to query molecules only in reagents
reactions_with_similar_reagent = db.Molecule.find_reactions_by_similar_reagent(query)
# reactions with similar to query molecules only in products
reactions_with_similar_product = db.Molecule.find_reactions_by_similar_product(query)


In [33]:
print(reactions_by_molecule[:5]) # list of ReactionContainers

[<CGRtools.containers.reaction.ReactionContainer object at 0x7f50fbd8e2e8>, <CGRtools.containers.reaction.ReactionContainer object at 0x7f50fbd8e288>, <CGRtools.containers.reaction.ReactionContainer object at 0x7f50fbd8e0a8>, <CGRtools.containers.reaction.ReactionContainer object at 0x7f50fbd8e048>, <CGRtools.containers.reaction.ReactionContainer object at 0x7f50fbd8e588>]


In [34]:
reactions_by_molecule[5]

In [35]:
reactions_with_similar_product[:5] 
# list of tuples (Reaction, Tanimoto(similarity of query to molecule in found reaction))

[(<CGRtools.containers.reaction.ReactionContainer at 0x7f50ef966d68>, 1.0)]

# Reaction Search

1. Structure search

In [28]:
query = RDFread('reactions.rdf').read()[0]
with db_session:
    reaction = db.Reaction.find_structure(query)

In [29]:
reaction, query

(Reaction[1],
 <CGRtools.containers.reaction.ReactionContainer at 0x7f50fbd8e1c8>)